Basic imports and path management

In [21]:
import pathlib
import sys
import time
import glob
from tqdm import tqdm
import multiprocessing

root = pathlib.Path("../../").resolve()
sys.path.append(str(root.joinpath("src", "python_framework_v02")))
sys.path.append(str(root.joinpath("src", "python_framework_v01")))
sys.path.append(str(root.joinpath("src", "python_routing_v01")))
sys.path.append(".")
import nhd_io as nio
import compute_nhd_routing_SingleSeg as tr
import nhd_network_utilities_v01 as nnu
import nhd_reach_utilities as nru

custom_input_folder = root.joinpath("test", "input", "yaml")
custom_input_file = "florence_933020089_dt60.yaml"
run_pocono2_test = None


Read the primary data input from json

In [22]:
supernetwork_parameters = None
waterbody_parameters = None
if custom_input_file:
    (
        supernetwork_parameters,
        waterbody_parameters,
        forcing_parameters,
        restart_parameters,
        output_parameters,
        run_parameters,
    ) = nio.read_custom_input(custom_input_folder.joinpath(custom_input_file))

    break_network_at_waterbodies = run_parameters.get(
        "break_network_at_waterbodies", None
    )

    dt = run_parameters.get("dt", None)
    nts = run_parameters.get("nts", None)
    qts_subdivisions = run_parameters.get("qts_subdivisions", None)
    debuglevel = -1 * int(run_parameters.get("debuglevel", 0))
    verbose = run_parameters.get("verbose", None)
    showtiming = run_parameters.get("showtiming", None)
    percentage_complete = run_parameters.get("percentage_complete", None)
    do_network_analysis_only = run_parameters.get("do_network_analysis_only", None)
    assume_short_ts = run_parameters.get("assume_short_ts", None)
    parallel_compute = run_parameters.get("parallel_compute", None)
    cpu_pool = run_parameters.get("cpu_pool", None)
    sort_networks = run_parameters.get("sort_networks", None)

    csv_output = output_parameters.get("csv_output", None)
    nc_output_folder = output_parameters.get("nc_output_folder", None)

    qlat_const = forcing_parameters.get("qlat_const", None)
    qlat_input_file = forcing_parameters.get("qlat_input_file", None)
    qlat_input_folder = forcing_parameters.get("qlat_input_folder", None)
    qlat_file_pattern_filter = forcing_parameters.get("qlat_file_pattern_filter", None)
    qlat_file_index_col = forcing_parameters.get("qlat_file_index_col", None)
    qlat_file_value_col = forcing_parameters.get("qlat_file_value_col", None)

    wrf_hydro_channel_restart_file = restart_parameters.get(
        "wrf_hydro_channel_restart_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file", None
    )
    wrf_hydro_channel_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_channel_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_channel_restart_upstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_upstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_downstream_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_downstream_flow_field_name", None
    )
    wrf_hydro_channel_restart_depth_flow_field_name = restart_parameters.get(
        "wrf_hydro_channel_restart_depth_flow_field_name", None
    )

    wrf_hydro_waterbody_restart_file = restart_parameters.get(
        "wrf_hydro_waterbody_restart_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file", None
    )
    wrf_hydro_waterbody_ID_crosswalk_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_ID_crosswalk_file_field_name", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file", None
    )
    wrf_hydro_waterbody_crosswalk_filter_file_field_name = restart_parameters.get(
        "wrf_hydro_waterbody_crosswalk_filter_file_field_name", None
    )

# Any specific commandline arguments will override the file
# TODO: There are probably some pathological collisions that could
# arise from this ordering ... check these out.


/home/jacob.hreha/github/t-route/src/python_framework_v02/nhd_io.py:53: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(custom_file)


In [23]:
if run_pocono2_test:
    if verbose:
        print("running test case for Pocono_TEST2 domain")
    # Overwrite the following test defaults
    supernetwork = "Pocono_TEST2"
    break_network_at_waterbodies = False
    qts_subdivisions = 1  # change qts_subdivisions = 1 as  default
    dt = 300 / qts_subdivisions
    nts = 144 * qts_subdivisions
    csv_output = {"csv_output_folder": os.path.join(root, "test", "output", "text")}
    nc_output_folder = os.path.join(root, "test", "output", "text")
    # test 1. Take lateral flow from re-formatted wrf-hydro output from Pocono Basin simulation
    qlat_input_file = os.path.join(
        root, r"test/input/geo/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.csv"
    )


In [24]:
if showtiming:
    program_start_time = time.time()
if verbose:
    print(f"begin program t-route ...")

# STEP 1: Read the supernetwork dataset and build the connections graph
if verbose:
    print("creating supernetwork connections set")
if showtiming:
    start_time = time.time()

if supernetwork_parameters:
    supernetwork_values = nnu.get_nhd_connections(
        supernetwork_parameters=supernetwork_parameters,
        verbose=False,
        debuglevel=debuglevel,
    )

else:
    test_folder = os.path.join(root, r"test")
    geo_input_folder = os.path.join(test_folder, r"input", r"geo")
    supernetwork_parameters, supernetwork_values = nnu.set_networks(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )
    waterbody_parameters = nnu.set_waterbody_parameters(
        supernetwork=supernetwork,
        geo_input_folder=geo_input_folder,
        verbose=False,
        debuglevel=debuglevel,
    )

if verbose:
    print("supernetwork connections set complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))

connections = supernetwork_values[0]


begin program t-route ...
creating supernetwork connections set
supernetwork connections set complete
... in 0.03888821601867676 seconds.


In [25]:
# STEP 2: Separate the networks and build the sub-graph of reaches within each network
if showtiming:
    start_time = time.time()
if verbose:
    print("organizing connections into reaches ...")
networks = nru.compose_networks(
    supernetwork_values,
    break_network_at_waterbodies=break_network_at_waterbodies,
    verbose=False,
    debuglevel=debuglevel,
    showtiming=showtiming,
)

if verbose:
    print("reach organization complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


organizing connections into reaches ...
reach organization complete
... in 0.004379987716674805 seconds.


In [26]:
# STEP 3: Organize Network for Waterbodies
if break_network_at_waterbodies:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("reading waterbody parameter file ...")

    ## STEP 3a: Read waterbody parameter file
    waterbodies_values = supernetwork_values[12]
    waterbodies_segments = supernetwork_values[13]
    connections_tailwaters = supernetwork_values[4]

    waterbodies_df = nio.read_waterbody_df(waterbody_parameters, waterbodies_values,)
    waterbodies_df = waterbodies_df.sort_index(axis="index").sort_index(axis="columns")

    nru.order_networks(connections, networks, connections_tailwaters)

    if verbose:
        print("waterbodies complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

    ## STEP 3b: Order subnetworks above and below reservoirs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("ordering waterbody subnetworks ...")

    max_network_seqorder = -1
    for network in networks:
        max_network_seqorder = max(
            networks[network]["network_seqorder"], max_network_seqorder
        )
    ordered_networks = {}

    for terminal_segment, network in networks.items():
        if network["network_seqorder"] not in ordered_networks:
            ordered_networks[network["network_seqorder"]] = []
        ordered_networks[network["network_seqorder"]].append(
            (terminal_segment, network)
        )

    if verbose:
        print("ordering waterbody subnetworks complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

else:
    # If we are not splitting the networks, we can put them all in one order
    max_network_seqorder = 0
    ordered_networks = {}
    ordered_networks[0] = [
        (terminal_segment, network) for terminal_segment, network in networks.items()
    ]

if do_network_analysis_only:
    sys.exit()

if break_network_at_waterbodies:
    ## STEP 3c: Handle Waterbody Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting waterbody initial states ...")

    if wrf_hydro_waterbody_restart_file:

        waterbody_initial_states_df = nio.get_reservoir_restart_from_wrf_hydro(
            wrf_hydro_waterbody_restart_file,
            wrf_hydro_waterbody_ID_crosswalk_file,
            wrf_hydro_waterbody_ID_crosswalk_file_field_name,
            wrf_hydro_waterbody_crosswalk_filter_file,
            wrf_hydro_waterbody_crosswalk_filter_file_field_name,
        )
    else:
        # TODO: Consider adding option to read cold state from route-link file
        waterbody_initial_ds_flow_const = 0.0
        waterbody_initial_depth_const = 0.0
        # Set initial states from cold-state
        waterbody_initial_states_df = pd.DataFrame(
            0, index=waterbodies_df.index, columns=["qd0", "h0",], dtype="float32"
        )
        # TODO: This assignment could probably by done in the above call
        waterbody_initial_states_df["qd0"] = waterbody_initial_ds_flow_const
        waterbody_initial_states_df["h0"] = waterbody_initial_depth_const
        waterbody_initial_states_df["index"] = range(len(waterbody_initial_states_df))

    if verbose:
        print("waterbody initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


reading waterbody parameter file ...
waterbodies complete
... in 0.017271041870117188 seconds.
ordering waterbody subnetworks ...
ordering waterbody subnetworks complete
... in 3.7670135498046875e-05 seconds.
setting waterbody initial states ...
waterbody initial states complete
... in 0.01991748809814453 seconds.


In [27]:
# STEP 4: Handle Channel Initial States
if showtiming:
    start_time = time.time()
if verbose:
    print("setting channel initial states ...")

if wrf_hydro_channel_restart_file:

    channel_initial_states_df = nio.get_stream_restart_from_wrf_hydro(
        wrf_hydro_channel_restart_file,
        wrf_hydro_channel_ID_crosswalk_file,
        wrf_hydro_channel_ID_crosswalk_file_field_name,
        wrf_hydro_channel_restart_upstream_flow_field_name,
        wrf_hydro_channel_restart_downstream_flow_field_name,
        wrf_hydro_channel_restart_depth_flow_field_name,
    )
else:
    # TODO: Consider adding option to read cold state from route-link file
    channel_initial_us_flow_const = 0.0
    channel_initial_ds_flow_const = 0.0
    channel_initial_depth_const = 0.0
    # Set initial states from cold-state
    channel_initial_states_df = pd.DataFrame(
        0, index=connections.keys(), columns=["qu0", "qd0", "h0",], dtype="float32"
    )
    channel_initial_states_df["qu0"] = channel_initial_us_flow_const
    channel_initial_states_df["qd0"] = channel_initial_ds_flow_const
    channel_initial_states_df["h0"] = channel_initial_depth_const
    channel_initial_states_df["index"] = range(len(channel_initial_states_df))

if verbose:
    print("channel initial states complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


setting channel initial states ...
channel initial states complete
... in 0.02246999740600586 seconds.


In [28]:
# STEP 5: Read (or set) QLateral Inputs
if showtiming:
    start_time = time.time()
if verbose:
    print("creating qlateral array ...")

# initialize qlateral dict
qlateral = {}

if qlat_input_folder:
    qlat_files = []
    for pattern in qlat_file_pattern_filter:
        qlat_files.extend(glob.glob(qlat_input_folder + pattern))
    qlat_df = nio.get_ql_from_wrf_hydro(
        qlat_files=qlat_files,
        index_col=qlat_file_index_col,
        value_col=qlat_file_value_col,
    )

elif qlat_input_file:
    qlat_df = nio.get_ql_from_csv(qlat_input_file)

else:
    qlat_df = pd.DataFrame(
        qlat_const, index=connections.keys(), columns=range(nts), dtype="float32"
    )

for index, row in qlat_df.iterrows():
    qlateral[index] = row.tolist()

if verbose:
    print("qlateral array complete")
if showtiming:
    print("... in %s seconds." % (time.time() - start_time))
    start_time = time.time()


creating qlateral array ...
qlateral array complete
... in 6.396091938018799 seconds.


In [29]:
# STEP 6: Sort the ordered networks
if sort_networks:
    if showtiming:
        start_time = time.time()
    if verbose:
        print("sorting the ordered networks ...")

    for nsq in range(max_network_seqorder, -1, -1):
        sort_ordered_network(ordered_networks[nsq], True)

    if verbose:
        print("sorting complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()


In [30]:
# Define the pool after we create the static global objects (and collect the garbage)
if parallel_compute:
    import gc

    gc.collect()
    pool = multiprocessing.Pool(cpu_pool)

flowveldepth_connect = (
    {}
)  # dict to contain values to transfer from upstream to downstream networks


writing segment output to --> ../../test/output/text/933020089.csv


In [31]:
################### Main Execution Loop across ordered networks
if showtiming:
    main_start_time = time.time()
if verbose:
    print(f"executing routing computation ...")

compute_network_func = tr.compute_network

tr.connections_g = connections
tr.networks_g = networks
tr.qlateral_g = qlateral
tr.waterbodies_df_g = waterbodies_df
tr.waterbody_initial_states_df_g = waterbody_initial_states_df
tr.channel_initial_states_df_g = channel_initial_states_df

progress_count = 0
percentage_complete = True
if percentage_complete:
    for nsq in range(max_network_seqorder, -1, -1):
        for terminal_segment, network in ordered_networks[nsq]:
            progress_count += len(network["all_segments"])
    pbar = tqdm(total=(progress_count))

for nsq in range(max_network_seqorder, -1, -1):

    if parallel_compute:
        nslist = []
    results = []

    current_index_total = 0

    for terminal_segment, network in ordered_networks[nsq]:

        if percentage_complete:
            if current_index_total == 0:
                pbar.update(0)

        if break_network_at_waterbodies:
            waterbody = waterbodies_segments.get(terminal_segment)
        else:
            waterbody = None
        if not parallel_compute:  # serial execution
            if showtiming:
                start_time = time.time()
            if verbose:
                print(
                    f"routing ordered reaches for terminal segment {terminal_segment} ..."
                )

            results.append(
                compute_network_func(
                    flowveldepth_connect=flowveldepth_connect,
                    terminal_segment=terminal_segment,
                    supernetwork_parameters=supernetwork_parameters,
                    waterbody_parameters=waterbody_parameters,
                    waterbody=waterbody,
                    nts=nts,
                    dt=dt,
                    qts_subdivisions=qts_subdivisions,
                    verbose=verbose,
                    debuglevel=debuglevel,
                    csv_output=csv_output,
                    nc_output_folder=nc_output_folder,
                    assume_short_ts=assume_short_ts,
                )
            )

            if showtiming:
                print("... complete in %s seconds." % (time.time() - start_time))
            if percentage_complete:
                pbar.update(len(network["all_segments"]))

        else:  # parallel execution
            nslist.append(
                [
                    flowveldepth_connect,
                    terminal_segment,
                    supernetwork_parameters,  # TODO: This should probably be global...
                    waterbody_parameters,
                    waterbody,
                    nts,
                    dt,
                    qts_subdivisions,
                    verbose,
                    debuglevel,
                    csv_output,
                    nc_output_folder,
                    assume_short_ts,
                ]
            )

    if parallel_compute:
        if verbose:
            print(f"routing ordered reaches for networks of order {nsq} ... ")
        if debuglevel <= -2:
            print(f"reaches to be routed include:")
            print(f"{[network[0] for network in ordered_networks[nsq]]}")
        # with pool:
        # with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_network_func, nslist)

        if showtiming:
            print("... complete in %s seconds." % (time.time() - start_time))
        if percentage_complete:
            pbar.update(
                sum(
                    len(network[1]["all_segments"]) for network in ordered_networks[nsq]
                )
            )
            # print(f"{[network[0] for network in ordered_networks[nsq]]}")

    max_courant = 0
    maxa = []
    for result in results:
        for seg in result:
            maxa.extend(result[seg][:, 8:9])
    max_courant = max(maxa)
    print(f"max_courant: {max_courant}")

    if (
        nsq > 0
    ):  # We skip this step for zero-order networks, i.e., those that have no downstream dependents
        flowveldepth_connect = (
            {}
        )  # There is no need to preserve previously passed on values -- so we clear the dictionary
        for i, (terminal_segment, network) in enumerate(ordered_networks[nsq]):
            # seg = network["reaches"][network["terminal_reach"]]["reach_tail"]
            seg = terminal_segment
            flowveldepth_connect[seg] = {}
            flowveldepth_connect[seg] = results[i][seg]
            # TODO: The value passed here could be much more specific to
            # TODO: exactly and only the most recent time step for the passing reach

if parallel_compute:
    pool.close()

if percentage_complete:
    pbar.close()

if verbose:
    print("ordered reach computation complete")
if showtiming:
    print("... in %s seconds." % (time.time() - main_start_time))
if verbose:
    print("program complete")
if showtiming:
    print("... in %s seconds." % (time.time() - program_start_time))



  0%|          | 0/1108 [00:00<?, ?it/s]

executing routing computation ...
routing ordered reaches for networks of order 2 ... 



 42%|████▏     | 467/1108 [00:35<00:48, 13.19it/s]

... complete in 35.58297324180603 seconds.
max_courant: [5.51265717]
routing ordered reaches for networks of order 1 ... 



 45%|████▌     | 499/1108 [00:49<01:53,  5.37it/s]

... complete in 49.79691505432129 seconds.
max_courant: [0.]
routing ordered reaches for networks of order 0 ... 



100%|██████████| 1108/1108 [01:50<00:00,  6.22it/s]

... complete in 111.17425584793091 seconds.


100%|██████████| 1108/1108 [01:54<00:00,  9.68it/s]

max_courant: [25.36383247]
ordered reach computation complete
... in 114.53386664390564 seconds.
program complete
... in 121.2260513305664 seconds.


In [32]:
# creation of the main dataframe


In [33]:
# print(results)
import numpy as np
import itertools
import pandas as pd

all_results = {}
seg_courant_maxes = []
time = []
flowval = []  # flowval
velval_list = []  # velval
depthval = []  # depthval
qlatval = []  # qlatval
storageval = []  # storageval
qlatCumval = []  # qlatCumval
kinCelerity = []  # ck
courant = []  # cn
X = []  # X

for result in results:
    all_results.update(result)

# print(all_results[8780801][1][0])

df = pd.DataFrame()
# print(all_results)

for key, data in all_results.items():
    time.extend(data[:, 0])  # time
    flowval.extend(data[:, 1])  # flowval
    velval_list.extend(data[:, 2])  # velval
    depthval.extend(data[:, 3])  # depthval
    qlatval.extend(data[:, 4])  # qlatval
    storageval.extend(data[:, 5])  # storageval
    qlatCumval.extend(data[:, 6])  # qlatCumval
    kinCelerity.extend(data[:, 7])  # ck
    courant.extend(data[:, 8])  # cn
    X.extend(data[:, 9])  # X


In [34]:
single_seg_length = len(all_results[933020089][:, 0])
single_seg_length * 611


8798400

In [35]:
len(time)


8798400

In [36]:
key_index = all_results.keys()
key_index = list(
    itertools.chain.from_iterable(
        itertools.repeat(x, single_seg_length) for x in key_index
    )
)
data = {
    "key_index": list(key_index),
    "time": list(time),
    "flowval": list(flowval),
    "velval_list": list(velval_list),
    "depthval": list(depthval),
    "qlatval": list(qlatval),
    "storageval": list(storageval),
    "qlatCumval": list(qlatCumval),
    "kinCelerity": list(kinCelerity),
    "courant": list(courant),
    "X": list(X),
}
df = pd.DataFrame(data)
df = df.set_index("key_index")
df = df.reset_index()
df = df.set_index("key_index")

df


,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
8780801,0.0,0.000180,0.034005,0.005051,0.000180,-3.929017e-08,1.079874e-02,0.056269,0.017676,0.499495
8780801,60.0,0.000180,0.034037,0.005058,0.000180,-7.508788e-08,2.159748e-02,0.056322,0.017693,0.499496
8780801,120.0,0.000180,0.034044,0.005060,0.000180,-1.082662e-07,3.239622e-02,0.056334,0.017696,0.499496
8780801,180.0,0.000180,0.034046,0.005060,0.000180,-1.388253e-07,4.319496e-02,0.056336,0.017697,0.499496
8780801,240.0,0.000180,0.034046,0.005060,0.000180,-1.667649e-07,5.399370e-02,0.056337,0.017697,0.499496
...,...,...,...,...,...,...,...,...,...,...
8778465,863700.0,0.000036,0.000000,0.000000,0.149393,8.135146e+06,1.368751e+06,0.000000,0.000000,0.000000
8778465,863760.0,0.000036,0.000000,0.000000,0.149393,8.135199e+06,1.368760e+06,0.000000,0.000000,0.000000
8778465,863820.0,0.000036,0.000000,0.000000,0.149393,8.135252e+06,1.368769e+06,0.000000,0.000000,0.000000


In [37]:
!pip install plotly

     |████████████████████████████████| 13.1 MB 6.9 MB/s eta 0:00:01
  Using cached retrying-1.3.3.tar.gz (10 kB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=955fe8c56e0da1a48727c21349a3bdf87e4a3b4377f30b8a6947f02fff9ed733
  Stored in directory: /home/jacob.hreha/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
You should consider upgrading via the '/home/jacob.hreha/anaconda3/bin/python -m pip install --upgrade pip' command.


In [38]:
# top 25 courants based on our work this morning and their segment IDs


In [39]:
seg_list = []
seg_courant_maxes = []
for seg in all_results:
    seg_list.append(seg)
    seg_courant_maxes.append(max(all_results[seg][:, tr.courant_index]))
zipped = zip(seg_list, seg_courant_maxes)
zipped = list(zipped)
res = sorted(zipped, key=lambda x: x[1], reverse=True)
# A.sort(reverse=True)
res = (res)[:25]
print((res)[:25])


[(933020027, 25.363832473754883), (8777735, 14.950478553771973), (933020017, 9.564001083374023), (8778581, 8.574037551879883), (933020059, 7.254691123962402), (933020061, 4.696855068206787), (8780597, 3.591726064682007), (933020020, 2.8707876205444336), (933020058, 2.4734983444213867), (933020023, 2.1059963703155518), (8777485, 1.860101580619812), (933020036, 1.6248382329940796), (8778795, 1.6150635480880737), (8778021, 1.4462398290634155), (8777339, 1.393180251121521), (8777867, 1.353600263595581), (8777477, 1.3393661975860596), (8780495, 1.3077224493026733), (8777861, 1.1583272218704224), (8777481, 1.149632215499878), (8780607, 1.0738232135772705), (8779003, 1.0580987930297852), (933020050, 1.0441739559173584), (8780751, 0.9917784929275513), (8778947, 0.9753884673118591)]


In [40]:
# A list of those IDs


In [41]:
major_segments = []
for x, y in res[:25]:
    major_segments.append(x)
major_segments


[933020027,
 8777735,
 933020017,
 8778581,
 933020059,
 933020061,
 8780597,
 933020020,
 933020058,
 933020023,
 8777485,
 933020036,
 8778795,
 8778021,
 8777339,
 8777867,
 8777477,
 8780495,
 8777861,
 8777481,
 8780607,
 8779003,
 933020050,
 8780751,
 8778947]

In [42]:
# Filtering out the above segment IDs from the original df.


In [43]:
df_filtered = df.loc[df.index.isin(major_segments), :]
df_filtered


,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
8777735,0.0,0.004196,0.185580,0.009796,0.000035,-6.722985e-08,0.002091,0.307107,1.675127,0.498455
8777735,60.0,0.004196,0.185576,0.009795,0.000035,-4.190952e-08,0.004183,0.307099,1.675087,0.498455
8777735,120.0,0.004196,0.185573,0.009795,0.000035,-4.627509e-08,0.006274,0.307094,1.675061,0.498455
8777735,180.0,0.004196,0.185570,0.009795,0.000035,-6.810296e-08,0.008365,0.307090,1.675035,0.498455
8777735,240.0,0.004196,0.185567,0.009794,0.000035,-7.770723e-08,0.010456,0.307085,1.675010,0.498455
...,...,...,...,...,...,...,...,...,...,...
8780751,863700.0,0.008624,0.134758,0.029789,0.000000,-1.558339e+01,50.799377,0.220286,0.145244,0.492948
8780751,863760.0,0.008624,0.134757,0.029788,0.000000,-1.558343e+01,50.799377,0.220284,0.145242,0.492948
8780751,863820.0,0.008623,0.134755,0.029788,0.000000,-1.558348e+01,50.799377,0.220281,0.145240,0.492948


In [44]:
df.loc[df["courant"] == 25.363832473754883]


,time,flowval,velval_list,depthval,qlatval,storageval,qlatCumval,kinCelerity,courant,X
key_index,,,,,,,,,,
933020027,561600.0,15.181515,4.458032,1.180036,0.0,58.131255,0.0,4.227305,25.363832,0.40232


In [45]:
# Automated plotter based on filtered dataframe


In [46]:
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.io as pio

df_indexed = df_filtered.reset_index()
test_chart = go.FigureWidget()
for x in range(0, 25):
    temp_df_range_1 = single_seg_length * (x)
    temp_df_range_2 = single_seg_length * (x + 1)
    print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment " + str(df_indexed["key_index"][temp_df_range_1]),
    )
test_chart.layout.title = "Timestep Chart " + str(dt)
plot(test_chart)


0 14400
14400 28800
28800 43200
43200 57600
57600 72000
72000 86400
86400 100800
100800 115200
115200 129600
129600 144000
144000 158400
158400 172800
172800 187200
187200 201600
201600 216000
216000 230400
230400 244800
244800 259200
259200 273600
273600 288000
288000 302400
302400 316800
316800 331200
331200 345600
345600 360000


'temp-plot.html'

In [47]:
custom_input_folder = root.joinpath("test", "input", "yaml")
custom_input_file = "florence_933020089_dt300.yaml"
all_results = tr.route_supernetwork(
    tr.connections_g,
    tr.networks_g,
    tr.qlateral_g,
    tr.waterbodies_df_g,
    tr.waterbody_initial_states_df_g,
    tr.channel_initial_states_df_g,
    custom_input_file=custom_input_folder.joinpath(custom_input_file),
)


/home/jacob.hreha/github/t-route/src/python_framework_v02/nhd_io.py:53: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



begin program t-route ...
creating supernetwork connections set
supernetwork connections set complete
... in 0.04615497589111328 seconds.
organizing connections into reaches ...
reach organization complete
... in 0.007277727127075195 seconds.
reading waterbody parameter file ...
waterbodies complete
... in 0.022621631622314453 seconds.
ordering waterbody subnetworks ...
ordering waterbody subnetworks complete
... in 4.4345855712890625e-05 seconds.
setting waterbody initial states ...
waterbody initial states complete
... in 0.0272982120513916 seconds.
setting channel initial states ...
channel initial states complete
... in 0.029682159423828125 seconds.
creating qlateral array ...
qlateral array complete
... in 8.037819862365723 seconds.
executing routing computation ...
routing ordered reaches for networks of order 2 ... 
... complete in 9.12061858177185 seconds.
max_courant: [27.53112221]
routing ordered reaches for networks of order 1 ... 
... complete in 11.953770399093628 seconds.

In [141]:
custom_input_file_list = ["florence_933020089_dt300.yaml","florence_933020089_dt60.yaml"]
from plotly.subplots import make_subplots
df_indexed_300 = pd.DataFrame()
df_indexed_60 = pd.DataFrame()
for c_i in (custom_input_file_list):
    print(c_i)
    import pathlib
    import sys
    import time
    import glob
    from tqdm import tqdm
    import multiprocessing

    root = pathlib.Path("../../").resolve()
    sys.path.append(str(root.joinpath("src", "python_framework_v02")))
    sys.path.append(str(root.joinpath("src", "python_framework_v01")))
    sys.path.append(str(root.joinpath("src", "python_routing_v01")))
    sys.path.append(".")
    import nhd_io as nio
    import compute_nhd_routing_SingleSeg as tr
    import nhd_network_utilities_v01 as nnu
    import nhd_reach_utilities as nru

    custom_input_folder = root.joinpath("test", "input", "yaml")
    custom_input_file = c_i
    run_pocono2_test = None

    supernetwork_parameters = None
    waterbody_parameters = None
    if custom_input_file:
        (
            supernetwork_parameters,
            waterbody_parameters,
            forcing_parameters,
            restart_parameters,
            output_parameters,
            run_parameters,
        ) = nio.read_custom_input(custom_input_folder.joinpath(custom_input_file))

        break_network_at_waterbodies = run_parameters.get(
            "break_network_at_waterbodies", None
        )

        dt = run_parameters.get("dt", None)
        nts = run_parameters.get("nts", None)
        qts_subdivisions = run_parameters.get("qts_subdivisions", None)
        debuglevel = -1 * int(run_parameters.get("debuglevel", 0))
        verbose = run_parameters.get("verbose", None)
        showtiming = run_parameters.get("showtiming", None)
        percentage_complete = run_parameters.get("percentage_complete", None)
        do_network_analysis_only = run_parameters.get("do_network_analysis_only", None)
        assume_short_ts = run_parameters.get("assume_short_ts", None)
        parallel_compute = run_parameters.get("parallel_compute", None)
        cpu_pool = run_parameters.get("cpu_pool", None)
        sort_networks = run_parameters.get("sort_networks", None)

        csv_output = output_parameters.get("csv_output", None)
        nc_output_folder = output_parameters.get("nc_output_folder", None)

        qlat_const = forcing_parameters.get("qlat_const", None)
        qlat_input_file = forcing_parameters.get("qlat_input_file", None)
        qlat_input_folder = forcing_parameters.get("qlat_input_folder", None)
        qlat_file_pattern_filter = forcing_parameters.get("qlat_file_pattern_filter", None)
        qlat_file_index_col = forcing_parameters.get("qlat_file_index_col", None)
        qlat_file_value_col = forcing_parameters.get("qlat_file_value_col", None)

        wrf_hydro_channel_restart_file = restart_parameters.get(
            "wrf_hydro_channel_restart_file", None
        )
        wrf_hydro_channel_ID_crosswalk_file = restart_parameters.get(
            "wrf_hydro_channel_ID_crosswalk_file", None
        )
        wrf_hydro_channel_ID_crosswalk_file_field_name = restart_parameters.get(
            "wrf_hydro_channel_ID_crosswalk_file_field_name", None
        )
        wrf_hydro_channel_restart_upstream_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_upstream_flow_field_name", None
        )
        wrf_hydro_channel_restart_downstream_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_downstream_flow_field_name", None
        )
        wrf_hydro_channel_restart_depth_flow_field_name = restart_parameters.get(
            "wrf_hydro_channel_restart_depth_flow_field_name", None
        )

        wrf_hydro_waterbody_restart_file = restart_parameters.get(
            "wrf_hydro_waterbody_restart_file", None
        )
        wrf_hydro_waterbody_ID_crosswalk_file = restart_parameters.get(
            "wrf_hydro_waterbody_ID_crosswalk_file", None
        )
        wrf_hydro_waterbody_ID_crosswalk_file_field_name = restart_parameters.get(
            "wrf_hydro_waterbody_ID_crosswalk_file_field_name", None
        )
        wrf_hydro_waterbody_crosswalk_filter_file = restart_parameters.get(
            "wrf_hydro_waterbody_crosswalk_filter_file", None
        )
        wrf_hydro_waterbody_crosswalk_filter_file_field_name = restart_parameters.get(
            "wrf_hydro_waterbody_crosswalk_filter_file_field_name", None
        )

    # Any specific commandline arguments will override the file
    # TODO: There are probably some pathological collisions that could
    # arise from this ordering ... check these out.

    if run_pocono2_test:
        if verbose:
            print("running test case for Pocono_TEST2 domain")
        # Overwrite the following test defaults
        supernetwork = "Pocono_TEST2"
        break_network_at_waterbodies = False
        qts_subdivisions = 1  # change qts_subdivisions = 1 as  default
        dt = 300 / qts_subdivisions
        nts = 144 * qts_subdivisions
        csv_output = {"csv_output_folder": os.path.join(root, "test", "output", "text")}
        nc_output_folder = os.path.join(root, "test", "output", "text")
        # test 1. Take lateral flow from re-formatted wrf-hydro output from Pocono Basin simulation
        qlat_input_file = os.path.join(
            root, r"test/input/geo/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.csv"
        )
    if showtiming:
        program_start_time = time.time()
    if verbose:
        print(f"begin program t-route ...")

    # STEP 1: Read the supernetwork dataset and build the connections graph
    if verbose:
        print("creating supernetwork connections set")
    if showtiming:
        start_time = time.time()

    if supernetwork_parameters:
        supernetwork_values = nnu.get_nhd_connections(
            supernetwork_parameters=supernetwork_parameters,
            verbose=False,
            debuglevel=debuglevel,
        )

    else:
        test_folder = os.path.join(root, r"test")
        geo_input_folder = os.path.join(test_folder, r"input", r"geo")
        supernetwork_parameters, supernetwork_values = nnu.set_networks(
            supernetwork=supernetwork,
            geo_input_folder=geo_input_folder,
            verbose=False,
            debuglevel=debuglevel,
        )
        waterbody_parameters = nnu.set_waterbody_parameters(
            supernetwork=supernetwork,
            geo_input_folder=geo_input_folder,
            verbose=False,
            debuglevel=debuglevel,
        )

    if verbose:
        print("supernetwork connections set complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

    connections = supernetwork_values[0]
    # STEP 2: Separate the networks and build the sub-graph of reaches within each network
    if showtiming:
        start_time = time.time()
    if verbose:
        print("organizing connections into reaches ...")
    networks = nru.compose_networks(
        supernetwork_values,
        break_network_at_waterbodies=break_network_at_waterbodies,
        verbose=False,
        debuglevel=debuglevel,
        showtiming=showtiming,
    )

    if verbose:
        print("reach organization complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 3: Organize Network for Waterbodies
    if break_network_at_waterbodies:
        if showtiming:
            start_time = time.time()
        if verbose:
            print("reading waterbody parameter file ...")

        ## STEP 3a: Read waterbody parameter file
        waterbodies_values = supernetwork_values[12]
        waterbodies_segments = supernetwork_values[13]
        connections_tailwaters = supernetwork_values[4]

        waterbodies_df = nio.read_waterbody_df(waterbody_parameters, waterbodies_values,)
        waterbodies_df = waterbodies_df.sort_index(axis="index").sort_index(axis="columns")

        nru.order_networks(connections, networks, connections_tailwaters)

        if verbose:
            print("waterbodies complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()

        ## STEP 3b: Order subnetworks above and below reservoirs
        if showtiming:
            start_time = time.time()
        if verbose:
            print("ordering waterbody subnetworks ...")

        max_network_seqorder = -1
        for network in networks:
            max_network_seqorder = max(
                networks[network]["network_seqorder"], max_network_seqorder
            )
        ordered_networks = {}

        for terminal_segment, network in networks.items():
            if network["network_seqorder"] not in ordered_networks:
                ordered_networks[network["network_seqorder"]] = []
            ordered_networks[network["network_seqorder"]].append(
                (terminal_segment, network)
            )

        if verbose:
            print("ordering waterbody subnetworks complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()

    else:
        # If we are not splitting the networks, we can put them all in one order
        max_network_seqorder = 0
        ordered_networks = {}
        ordered_networks[0] = [
            (terminal_segment, network) for terminal_segment, network in networks.items()
        ]

    if do_network_analysis_only:
        sys.exit()

    if break_network_at_waterbodies:
        ## STEP 3c: Handle Waterbody Initial States
        if showtiming:
            start_time = time.time()
        if verbose:
            print("setting waterbody initial states ...")

        if wrf_hydro_waterbody_restart_file:

            waterbody_initial_states_df = nio.get_reservoir_restart_from_wrf_hydro(
                wrf_hydro_waterbody_restart_file,
                wrf_hydro_waterbody_ID_crosswalk_file,
                wrf_hydro_waterbody_ID_crosswalk_file_field_name,
                wrf_hydro_waterbody_crosswalk_filter_file,
                wrf_hydro_waterbody_crosswalk_filter_file_field_name,
            )
        else:
            # TODO: Consider adding option to read cold state from route-link file
            waterbody_initial_ds_flow_const = 0.0
            waterbody_initial_depth_const = 0.0
            # Set initial states from cold-state
            waterbody_initial_states_df = pd.DataFrame(
                0, index=waterbodies_df.index, columns=["qd0", "h0",], dtype="float32"
            )
            # TODO: This assignment could probably by done in the above call
            waterbody_initial_states_df["qd0"] = waterbody_initial_ds_flow_const
            waterbody_initial_states_df["h0"] = waterbody_initial_depth_const
            waterbody_initial_states_df["index"] = range(len(waterbody_initial_states_df))

        if verbose:
            print("waterbody initial states complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()
    # STEP 4: Handle Channel Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting channel initial states ...")

    if wrf_hydro_channel_restart_file:

        channel_initial_states_df = nio.get_stream_restart_from_wrf_hydro(
            wrf_hydro_channel_restart_file,
            wrf_hydro_channel_ID_crosswalk_file,
            wrf_hydro_channel_ID_crosswalk_file_field_name,
            wrf_hydro_channel_restart_upstream_flow_field_name,
            wrf_hydro_channel_restart_downstream_flow_field_name,
            wrf_hydro_channel_restart_depth_flow_field_name,
        )
    else:
        # TODO: Consider adding option to read cold state from route-link file
        channel_initial_us_flow_const = 0.0
        channel_initial_ds_flow_const = 0.0
        channel_initial_depth_const = 0.0
        # Set initial states from cold-state
        channel_initial_states_df = pd.DataFrame(
            0, index=connections.keys(), columns=["qu0", "qd0", "h0",], dtype="float32"
        )
        channel_initial_states_df["qu0"] = channel_initial_us_flow_const
        channel_initial_states_df["qd0"] = channel_initial_ds_flow_const
        channel_initial_states_df["h0"] = channel_initial_depth_const
        channel_initial_states_df["index"] = range(len(channel_initial_states_df))

    if verbose:
        print("channel initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 5: Read (or set) QLateral Inputs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("creating qlateral array ...")

    # initialize qlateral dict
    qlateral = {}

    if qlat_input_folder:
        qlat_files = []
        for pattern in qlat_file_pattern_filter:
            qlat_files.extend(glob.glob(qlat_input_folder + pattern))
        qlat_df = nio.get_ql_from_wrf_hydro(
            qlat_files=qlat_files,
            index_col=qlat_file_index_col,
            value_col=qlat_file_value_col,
        )

    elif qlat_input_file:
        qlat_df = nio.get_ql_from_csv(qlat_input_file)

    else:
        qlat_df = pd.DataFrame(
            qlat_const, index=connections.keys(), columns=range(nts), dtype="float32"
        )

    for index, row in qlat_df.iterrows():
        qlateral[index] = row.tolist()

    if verbose:
        print("qlateral array complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()
    # STEP 6: Sort the ordered networks
    if sort_networks:
        if showtiming:
            start_time = time.time()
        if verbose:
            print("sorting the ordered networks ...")

        for nsq in range(max_network_seqorder, -1, -1):
            sort_ordered_network(ordered_networks[nsq], True)

        if verbose:
            print("sorting complete")
        if showtiming:
            print("... in %s seconds." % (time.time() - start_time))
            start_time = time.time()
    # Define the pool after we create the static global objects (and collect the garbage)
    if parallel_compute:
        import gc

        gc.collect()
        pool = multiprocessing.Pool(cpu_pool)

    flowveldepth_connect = (
        {}
    )  # dict to contain values to transfer from upstream to downstream networks
    ################### Main Execution Loop across ordered networks
    if showtiming:
        main_start_time = time.time()
    if verbose:
        print(f"executing routing computation ...")

    compute_network_func = tr.compute_network

    tr.connections_g = connections
    tr.networks_g = networks
    tr.qlateral_g = qlateral
    tr.waterbodies_df_g = waterbodies_df
    tr.waterbody_initial_states_df_g = waterbody_initial_states_df
    tr.channel_initial_states_df_g = channel_initial_states_df

    progress_count = 0
    percentage_complete = True
    if percentage_complete:
        for nsq in range(max_network_seqorder, -1, -1):
            for terminal_segment, network in ordered_networks[nsq]:
                progress_count += len(network["all_segments"])
        pbar = tqdm(total=(progress_count))

    for nsq in range(max_network_seqorder, -1, -1):

        if parallel_compute:
            nslist = []
        results = []

        current_index_total = 0

        for terminal_segment, network in ordered_networks[nsq]:

            if percentage_complete:
                if current_index_total == 0:
                    pbar.update(0)

            if break_network_at_waterbodies:
                waterbody = waterbodies_segments.get(terminal_segment)
            else:
                waterbody = None
            if not parallel_compute:  # serial execution
                if showtiming:
                    start_time = time.time()
                if verbose:
                    print(
                        f"routing ordered reaches for terminal segment {terminal_segment} ..."
                    )

                results.append(
                    compute_network_func(
                        flowveldepth_connect=flowveldepth_connect,
                        terminal_segment=terminal_segment,
                        supernetwork_parameters=supernetwork_parameters,
                        waterbody_parameters=waterbody_parameters,
                        waterbody=waterbody,
                        nts=nts,
                        dt=dt,
                        qts_subdivisions=qts_subdivisions,
                        verbose=verbose,
                        debuglevel=debuglevel,
                        csv_output=csv_output,
                        nc_output_folder=nc_output_folder,
                        assume_short_ts=assume_short_ts,
                    )
                )

                if showtiming:
                    print("... complete in %s seconds." % (time.time() - start_time))
                if percentage_complete:
                    pbar.update(len(network["all_segments"]))

            else:  # parallel execution
                nslist.append(
                    [
                        flowveldepth_connect,
                        terminal_segment,
                        supernetwork_parameters,  # TODO: This should probably be global...
                        waterbody_parameters,
                        waterbody,
                        nts,
                        dt,
                        qts_subdivisions,
                        verbose,
                        debuglevel,
                        csv_output,
                        nc_output_folder,
                        assume_short_ts,
                    ]
                )

        if parallel_compute:
            if verbose:
                print(f"routing ordered reaches for networks of order {nsq} ... ")
            if debuglevel <= -2:
                print(f"reaches to be routed include:")
                print(f"{[network[0] for network in ordered_networks[nsq]]}")
            # with pool:
            # with multiprocessing.Pool() as pool:
            results = pool.starmap(compute_network_func, nslist)

            if showtiming:
                print("... complete in %s seconds." % (time.time() - start_time))
            if percentage_complete:
                pbar.update(
                    sum(
                        len(network[1]["all_segments"]) for network in ordered_networks[nsq]
                    )
                )
                # print(f"{[network[0] for network in ordered_networks[nsq]]}")

        max_courant = 0
        maxa = []
        for result in results:
            for seg in result:
                maxa.extend(result[seg][:, 8:9])
        max_courant = max(maxa)
        print(f"max_courant: {max_courant}")

        if (
            nsq > 0
        ):  # We skip this step for zero-order networks, i.e., those that have no downstream dependents
            flowveldepth_connect = (
                {}
            )  # There is no need to preserve previously passed on values -- so we clear the dictionary
            for i, (terminal_segment, network) in enumerate(ordered_networks[nsq]):
                # seg = network["reaches"][network["terminal_reach"]]["reach_tail"]
                seg = terminal_segment
                flowveldepth_connect[seg] = {}
                flowveldepth_connect[seg] = results[i][seg]
                # TODO: The value passed here could be much more specific to
                # TODO: exactly and only the most recent time step for the passing reach

    if parallel_compute:
        pool.close()

    if percentage_complete:
        pbar.close()

    if verbose:
        print("ordered reach computation complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - main_start_time))
    if verbose:
        print("program complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - program_start_time))
    
    
    
    all_results_new = {}
    seg_courant_maxes = []
    time = []
    flowval = []  # flowval
    velval_list = []  # velval
    depthval = []  # depthval
    qlatval = []  # qlatval
    storageval = []  # storageval
    qlatCumval = []  # qlatCumval
    kinCelerity = []  # ck
    courant = []  # cn
    X = []  # X

    for result in results:
        all_results_new.update(result)
    
    # print(all_results[8780801][1][0])

    df = pd.DataFrame()
    # print(all_results)

    for key, data in all_results_new.items():
        time.extend(data[:, 0])  # time
        flowval.extend(data[:, 1])  # flowval
        velval_list.extend(data[:, 2])  # velval
        depthval.extend(data[:, 3])  # depthval
        qlatval.extend(data[:, 4])  # qlatval
        storageval.extend(data[:, 5])  # storageval
        qlatCumval.extend(data[:, 6])  # qlatCumval
        kinCelerity.extend(data[:, 7])  # ck
        courant.extend(data[:, 8])  # cn
        X.extend(data[:, 9])  # X
#     print(all_results_new)
    single_seg_length = len(all_results_new[933020089][:, 0])

    if c_i == "florence_933020089_dt300.yaml":
        single_seg_length_300 = single_seg_length
    else:
        single_seg_length_60 = single_seg_length
#     else:
#         single_seg_length_10 = single_seg_length
#     print(single_seg_length_300,single_seg_length_60)
    # single_seg_length * 611

    key_index = all_results.keys()
    key_index = list(
        itertools.chain.from_iterable(
            itertools.repeat(x, single_seg_length) for x in key_index
        )
    )
    data = {
        "key_index": list(key_index),
        "time": list(time),
        "flowval": list(flowval),
        "velval_list": list(velval_list),
        "depthval": list(depthval),
        "qlatval": list(qlatval),
        "storageval": list(storageval),
        "qlatCumval": list(qlatCumval),
        "kinCelerity": list(kinCelerity),
        "courant": list(courant),
        "X": list(X),
    }
    df = pd.DataFrame(data)
    df = df.set_index("key_index")
    df = df.reset_index()
    df = df.set_index("key_index")


    seg_list = []
    seg_courant_maxes = []
    for seg in all_results_new:
        seg_list.append(seg)
        seg_courant_maxes.append(max(all_results_new[seg][:, tr.courant_index]))
    zipped = zip(seg_list, seg_courant_maxes)
    zipped = list(zipped)
    res = sorted(zipped, key=lambda x: x[1], reverse=True)
    # A.sort(reverse=True)
    res = (res)[:3]
    # print((res)[:25])

    major_segments = []
    for x, y in res[:3]:
        major_segments.append(x)
    # major_segments

    df_filtered = df.loc[df.index.isin(major_segments), :]

    if c_i == "florence_933020089_dt300.yaml":
        df_indexed_300 = df_filtered.reset_index()
    else:
        df_indexed_60 = df_filtered.reset_index()
    print(df_indexed_300,df_indexed_60)
#     else:
#         df_indexed_10 = df_filtered.reset_index()

test_chart = make_subplots(specs=[[{"secondary_y": True}]])
# test_chart = go.FigureWidget()
for x in range(0,len(major_segments)):
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment flowval " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),
    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["flowval"],
        name="segment flowval " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),
    )
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["courant"],secondary_y=True,
        name="segment courant " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),
    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["courant"],secondary_y=True,
        name="segment courant " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),
    )
# test_chart.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
# test_chart.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

test_chart.layout.title = "Timestep Chart 300, 60"
plot(test_chart)

florence_933020089_dt300.yaml
begin program t-route ...
creating supernetwork connections set
supernetwork connections set complete
... in 0.03426241874694824 seconds.
organizing connections into reaches ...
reach organization complete
... in 0.003715991973876953 seconds.
reading waterbody parameter file ...
waterbodies complete
... in 0.016068458557128906 seconds.
ordering waterbody subnetworks ...
ordering waterbody subnetworks complete
... in 3.0279159545898438e-05 seconds.
setting waterbody initial states ...
waterbody initial states complete
... in 0.017216205596923828 seconds.
setting channel initial states ...
channel initial states complete
... in 0.01768350601196289 seconds.
creating qlateral array ...


/home/jacob.hreha/github/t-route/src/python_framework_v02/nhd_io.py:53: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



qlateral array complete
... in 8.201142072677612 seconds.



  0%|          | 0/1108 [00:00<?, ?it/s]

executing routing computation ...
routing ordered reaches for networks of order 2 ... 


...

... complete in 112.29779148101807 seconds.


100%|██████████| 1108/1108 [01:55<00:00,  9.62it/s]

max_courant: [25.36383247]
ordered reach computation complete
... in 115.23613929748535 seconds.
program complete
... in 122.44014620780945 seconds.


      key_index      time   flowval  velval_list  depthval   qlatval  \
0       8777735       0.0  0.004196     0.185580  0.009796  0.000035   
1       8777735     300.0  0.004196     0.185566  0.009794  0.000035   
2       8777735     600.0  0.004195     0.185552  0.009793  0.000035   
3       8777735     900.0  0.004194     0.185539  0.009792  0.000035   
4       8777735    1200.0  0.004193     0.185527  0.009791  0.000035   
...         ...       ...       ...          ...       ...       ...   
8635  933020027  862500.0  0.058953     0.755009  0.049310  0.000000   
8636  933020027  862800.0  0.058909     0.754795  0.049287  0.000000   
8637  933020027  863100.0  0.058867     0.754590  0.049266  0.000000   
8638  933020027  863400.0  0.058826     0.754394  0.049246  0.000000   
8639  933020027  863700.0  0.058788     0.754206  0.049226  0.000000   

        storageval  qlatCumval  kinCelerity    courant         X  
0    -3.361492e-07    0.010456     0.307107   8.375633  0.498455  
1

'temp-plot.html'

In [147]:
import random
test_chart = make_subplots(specs=[[{"secondary_y": True}]])
# test_chart = go.FigureWidget()

for x in range(0,len(major_segments)):
    r = lambda: random.randint(0,255)
    temp_color1 = ('#%02X%02X%02X' % (r(),r(),r()))
    temp_color2 = ('#%02X%02X%02X' % (r(),r(),r()))
#     print(temp_color1,temp_color2)
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["flowval"],line_color=temp_color1,
        name="segment flowval " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),

    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["flowval"],line_color=temp_color2,
        name="segment flowval " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),

    )
    temp_df_range_1 = single_seg_length_300 * (x)
    temp_df_range_2 = single_seg_length_300 * (x + 1)
    test_chart.add_scatter(
        x=df_indexed_300[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_300[temp_df_range_1:temp_df_range_2]["courant"],line_color=temp_color1,secondary_y=True,
        name="segment courant " + str(df_indexed_300["key_index"][temp_df_range_1]) + " " + str(300),

    )
    temp_df_range_1 = single_seg_length_60 * (x)
    temp_df_range_2 = single_seg_length_60 * (x + 1)
    #         print(temp_df_range_1, temp_df_range_2)
    test_chart.add_scatter(
        x=df_indexed_60[temp_df_range_1:temp_df_range_2]["time"],
        y=df_indexed_60[temp_df_range_1:temp_df_range_2]["courant"],
        line_color=temp_color2,
        secondary_y=True,
        name="segment courant " + str(df_indexed_60["key_index"][temp_df_range_1]) + " " + str(60),
    )
test_chart.update_yaxes(title_text="Flow Value", secondary_y=False)
# test_chart.update_yaxes(title_text="Courant Value", secondary_y=True)
test_chart.update_xaxes(title_text="Time Step")
test_chart.layout.title = "Timestep Chart 300, 60"
plot(test_chart)

'temp-plot.html'

In [135]:
import random
r = lambda: random.randint(0,255)
gen1 = ('#%02X%02X%02X' % (r(),r(),r()))
gen1

'#A79870'

In [59]:
df_lengths = df_indexed_300
lengths_temp = []
for i, j in df_lengths.iterrows(): 
    lengths_temp.append(supernetwork_values[0][j['key_index']]['length'])
df_lengths['lengths'] = pd.Series(lengths_temp, index=df_lengths.index)

In [60]:
# df_lengths.index.unique()

In [61]:
df_lengths = df_lengths.reset_index(drop=True)
df_lengths = df_lengths.set_index('key_index')


In [62]:
len(major_segments)

25

In [63]:
length_list = []
courant_list = []
flowval_list = []
for i in major_segments:
    length_list.append(df_lengths.loc[i]['lengths'][:1].values[0])
    courant_list.append(df_lengths.loc[i]['courant'][:1].values[0])
    flowval_list.append(df_lengths.loc[i]['flowval'][:1].values[0])

In [64]:
courant_list

[12.227744102478027,
 8.375633239746094,
 1.4919893741607666,
 3.5384578704833984,
 2.722485065460205,
 4.67125129699707,
 1.958962321281433,
 1.6834832429885864,
 4.80889892578125,
 1.0520790815353394,
 0.5376074314117432,
 0.49882617592811584,
 0.17622336745262146,
 2.33585524559021,
 0.43409964442253113,
 1.903039813041687,
 0.3100087344646454,
 0.4152270555496216,
 1.5580898523330688,
 0.3247401714324951,
 0.6226049661636353,
 0.6581174731254578,
 0.735342800617218,
 0.35107898712158203,
 0.2029656171798706]

In [93]:
df_indexed_60['key_index']

0         8777735
1         8777735
2         8777735
3         8777735
4         8777735
           ...   
359995    8780751
359996    8780751
359997    8780751
359998    8780751
359999    8780751
Name: key_index, Length: 360000, dtype: int64

In [66]:
# table1 = zip(length_list,major_segments)
# table2 = sorted(table1, key=lambda x: x[1], reverse=True)

In [67]:
b = length_list
a = major_segments
c = courant_list
d = flowval_list
res = "\n".join("{}      {}      {}       {}".format(w   , x  , y   , z  ) for w, x, y, z in sorted(zip(c, d, b, a),reverse=True))

In [68]:
print(res)


12.227744102478027      0.004060585051774979      10.0       933020027
8.375633239746094      0.0041962782852351665      11.0       8777735
4.80889892578125      6.211640357971191      5.0       933020058
4.67125129699707      0.005805434659123421      17.0       933020061
3.5384578704833984      2.412903308868408      5.0       8778581
2.722485065460205      0.0026232698000967503      17.0       933020059
2.33585524559021      0.6656259894371033      114.0       8778021
1.958962321281433      0.008839867077767849      54.0       8780597
1.903039813041687      1.610382318496704      166.0       8777867
1.6834832429885864      0.0018756598001345992      28.0       933020020
1.5580898523330688      1.6185015439987183      177.0       8777861
1.4919893741607666      6.569494416908128e-06      11.0       933020017
1.0520790815353394      0.009235578589141369      57.0       933020023
0.735342800617218      0.0011249413946643472      46.0       933020050
0.6581174731254578      0.1430711448

In [82]:
test_chart = make_subplots(specs=[[{"secondary_y": True}]])
test_chart.add_scatter(
    x=length_list,
    y=courant_list,
    name="courant" + " " + str(300),
    mode='markers',  
)
test_chart.add_scatter(
    x=length_list,
    y=flowval_list,
    name="flowval" + " " + str(300),
    mode='markers',  
)
# test_chart.update_yaxes(title_text="Flow Value", secondary_y=False)
# test_chart.update_yaxes(title_text="Courant Value", secondary_y=True)
test_chart.update_xaxes(title_text="Length", )

test_chart.layout.title = "Top 25 Values by Length - 300 "
plot(test_chart)

'temp-plot.html'